### Sentiment Analysis in Italian using Transformers

based on Neuraly work:
* see: https://huggingface.co/neuraly/bert-base-italian-cased-sentiment
* https://medium.com/@a.bellini/leveraging-huggingfaces-transformers-for-cross-lingual-sentiment-analysis-acca1f4e9da6

In [1]:
import torch
import numpy as np
from torch import nn  
from transformers import AutoTokenizer, AutoModelForSequenceClassification

### Python class

I have encapsulated the code from the HF site of the model, with some semplification, in a Python class.

In [2]:
class ITASentimentAnalyzer:
    # load the tokenizer and transformer
    def __init__(self, MODEL_NAME):
        
        # for rounding
        self.DEC_DIGITS = 4
        
        # name of HuggingFace model used
        self.MODEL_NAME = MODEL_NAME
        
        print("Loading model...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.MODEL_NAME)
        # Load the model
        self.model = AutoModelForSequenceClassification.from_pretrained(self.MODEL_NAME)
        
        print("Model loading completed!")
    
    # utility to get rid of tensor and round
    def round(self, tens_val):
        
        return round(tens_val.item(), self.DEC_DIGITS)
    
    def compute_ids(self, input_sentence):
        input_ids = self.tokenizer(input_sentence, add_special_tokens=True)['input_ids']
        print(input_ids)
        
        return
    
    #
    # does the scoring on a single sentence a time
    #
    def score(self, input_sentence):
        # encode the sentence and create the input tensor (in PyTorch format)
        input_ids = self.tokenizer(input_sentence, add_special_tokens=True, return_tensors='pt')['input_ids']
        
        # output from tokenizer is already a tensor
        
        # Call the model and get the logits
        with torch.no_grad():
            logits = self.model(input_ids)['logits']

        # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
        proba = nn.functional.softmax(logits, dim=1)
    
        # to remove the added dimension with squeeze
        # proba is (negative, neutral, positive)
        scores = proba.squeeze(0)
        
        # get rid of tensor and round and 
        # prepare the output json
        
        ret_vet = []
        
        for i, label in enumerate(['negative', 'neutral', 'positive']):
            ret_vet.append({'label' : label, 'score' : self.round(scores[i])})
        
        return ret_vet
    
    def batch_predict(self, input_sentences):
        # encode the sentence and create the input tensor
        input_ids = self.tokenizer(input_sentences, add_special_tokens=True, padding=True)['input_ids']
        
        print(input_ids)
        
        return

In [3]:
%%time

# loading the model: pass the HF model name
sent_analyzer = ITASentimentAnalyzer("neuraly/bert-base-italian-cased-sentiment")

Loading model...
Model loading completed!
CPU times: user 827 ms, sys: 147 ms, total: 974 ms
Wall time: 5.95 s


In [4]:
scores = sent_analyzer.score("Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela")

scores

[{'label': 'negative', 'score': 0.9716},
 {'label': 'neutral', 'score': 0.0181},
 {'label': 'positive', 'score': 0.0103}]

### test su un set di frasi

In [5]:
%%time

input_sentences = [
    "E' un prodotto pessimo",
    "La sua organizzazione ha fornito un buon servizio alla clientela",
    "La sua organizzazione non ha fornito un buon servizio alla clientela",
    "Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela",
    "Il prodotto non funziona, non comprero' più nulla dalla vostra azienda",
    "Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona?",
    "La gestione da parte della Regione Lazio della complessa macchina dei vaccini è stata buona",
    "La vostra organizzazione offre servizi pessimi",
    "La vostra organizzazione offre servizi non adeguati",
    "Sono molto soddisfatto del tuo lavoro",
    "non sono del tutto sicuro che il lavoro sia adeguato",
    "l'azienda dovrebbe offrire servizi migliori",
    "il supporto offerto dal customer care non è stato adeguato",
    "il risultato è pessimo",
    "il Napoli ha giocato una partita ottima",
    "il lavoro dell'allenatore è stato modesto"
]


# sent_analyzer = ITASentimentAnalyzer()
for sentence in input_sentences:
        scores = sent_analyzer.score(sentence)
    
        print(sentence)
        print(scores)
        print()
            
# formatting
print()

E' un prodotto pessimo
[{'label': 'negative', 'score': 0.9978}, {'label': 'neutral', 'score': 0.002}, {'label': 'positive', 'score': 0.0002}]

La sua organizzazione ha fornito un buon servizio alla clientela
[{'label': 'negative', 'score': 0.0002}, {'label': 'neutral', 'score': 0.002}, {'label': 'positive', 'score': 0.9978}]

La sua organizzazione non ha fornito un buon servizio alla clientela
[{'label': 'negative', 'score': 0.9951}, {'label': 'neutral', 'score': 0.0046}, {'label': 'positive', 'score': 0.0003}]

Non credo che la sua organizzazione abbia fornito un buon servizio alla clientela
[{'label': 'negative', 'score': 0.9716}, {'label': 'neutral', 'score': 0.0181}, {'label': 'positive', 'score': 0.0103}]

Il prodotto non funziona, non comprero' più nulla dalla vostra azienda
[{'label': 'negative', 'score': 0.9976}, {'label': 'neutral', 'score': 0.0023}, {'label': 'positive', 'score': 0.0002}]

Io penso che la sua organizzazione non abbia fornito un buon servizio alla clientela
[{

### next step: Test batch